In [1]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import cv2
from PIL import Image
from skimage.transform import resize
from skimage.io import imsave
from sklearn.model_selection import train_test_split

In [3]:
import tensorflow as tf
import tensorflow
from tensorflow import keras
from keras.layers import Dense, Conv2D, Flatten

from keras import backend as K

Using TensorFlow backend.


In [4]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

In [5]:
K.set_image_data_format('channels_last')

In [6]:
data_path = "/home/root/share/ultrasound-nerve-segmentation.zip"

In [ ]:
!mkdir /home/root/share/ultrasound-nerve-segmentation-1

In [ ]:
!unzip /home/root/share/ultrasound-nerve-segmentation.zip -d /home/root/share/ultrasound-nerve-segmentation-1

In [ ]:
!unzip /home/root/share/ultrasound-nerve-segmentation-1/train.zip 
!unzip /home/root/share/ultrasound-nerve-segmentation-1/test.zip

In [ ]:
!ls /home/root/share

In [7]:
data_path = "/home/root/share/ultrasound-nerve-segmentation-1/"

In [ ]:
!mkdir /home/root/share/ultrasound-nerve-segmentation-1/train_1
!mkdir /home/root/share/ultrasound-nerve-segmentation-1/test_1

!unzip /home/root/share/ultrasound-nerve-segmentation-1/train.zip -d /home/root/share/ultrasound-nerve-segmentation-1/train_1
!unzip /home/root/share/ultrasound-nerve-segmentation-1/test.zip -d /home/root/share/ultrasound-nerve-segmentation-1/test_1

In the train folder we have images and masks. Let us get them into seperate lists to hel us in the training.

In [8]:
train_data_path = "/home/root/share/ultrasound-nerve-segmentation-1/train_1/train/"
test_data_path = "/home/root/share/ultrasound-nerve-segmentation-1/test_1/test/"

In [9]:
def get_train_test_data(train_data_path, test_data_path):
    train_data_files = os.listdir(train_data_path)
    train = []
    train_mask = []
    train_files = []
    test_id = []
    #adding images and masks to seperate lists.
    for image_file_name in train_data_files:
        if 'mask' in image_file_name:
            continue
        
        image_mask_name = image_file_name.split('.')[0] + '_mask.tif'
        train.append((np.array(Image.open(train_data_path + image_file_name))))
        train_mask.append((np.array(Image.open(train_data_path + image_mask_name))))
        train_files.append(image_file_name)
        
    #Handling test data
    test_data_files = os.listdir(test_data_path)
    test = []    
    for image_file_name in test_data_files:
        test.append((np.array(Image.open(test_data_path + image_file_name))))
        test_id.append(int(image_file_name.split('.')[0]))
        
    return train, train_mask ,test, train_files, test_id

In [10]:
X_train , y_train , X_test, train_file_names, test_id = get_train_test_data(train_data_path, test_data_path)

lets check for a random image

In [ ]:
index = 28

image1 = X_train[index]
image1_mask  = y_train[index]
orig_file_name = train_file_names[index]
image1_orig = np.array(Image.open(train_data_path + orig_file_name)) 
image1_orig_mask = np.array(Image.open(train_data_path + orig_file_name.split('.')[0] + '_mask.tif'))
fig, ax = plt.subplots(2,2,figsize = (16,12))
ax[0][0].imshow(image1, cmap = 'gray')
ax[0][1].imshow(image1_mask, cmap = 'gray')
ax[1][0].imshow(image1_orig, cmap = 'gray')
ax[1][1].imshow(image1_orig_mask, cmap = 'gray')

In [ ]:
np.array(X_train).shape

Lets check how to view train_mask.csv

In [11]:
mask_df = pd.read_csv("/home/root/share/ultrasound-nerve-segmentation-1/train_masks.csv")
mask_df.head()

,subject,img,pixels
0,1,1,168153 9 168570 15 168984 22 169401 26 169818 ...
1,1,2,96346 7 96762 16 97179 27 97595 35 98012 40 98...
2,1,3,121957 2 122365 26 122784 28 123202 31 123618 ...
3,1,4,110980 9 111393 21 111810 33 112228 43 112647 ...
4,1,5,NaN


In [12]:
image_width = 400
image_height = 400

In [13]:
def preprocess_img_data(X_train , y_train , X_test):
    num_train_images = np.array(X_train).shape[0]
    img_width = image_width
    img_height = image_height
    
    imgs = np.ndarray((num_train_images, image_height, image_width), dtype=np.uint8)
    imgs_mask = np.ndarray((num_train_images, image_height, image_width), dtype=np.uint8)
    
    for i in range(num_train_images):
        imgs[i] = resize(X_train[i], (img_width, img_height), preserve_range=True)
        imgs_mask[i] = resize(y_train[i], (img_width, img_height), preserve_range=True)

    imgs = imgs[..., np.newaxis]
    imgs_mask = imgs_mask[..., np.newaxis]
    
    num_test_images = np.array(X_test).shape[0]    
    imgs_test =  np.ndarray((num_test_images, image_height, image_width), dtype=np.uint8)
    
    for i in range(num_test_images):
        imgs_test[i] = resize(X_test[i], (img_width, img_height), preserve_range=True)
    
    imgs_test = imgs_test[..., np.newaxis]
    
    #data normalization , mean centric
    imgs_train = imgs.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization
    
    imgs_train -= mean
    imgs_train /= std
    
    imgs_mask_train = imgs_mask.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]
    
    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std
    
    return imgs_train, imgs_mask_train, imgs_test

In [14]:
def dice_coef(y_true , y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth) 

In [15]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [16]:
def add2Conv2D_blocks(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x
    

def unet_model(height, width, n_filters = 16 , dropout = 0.1):
    
    input_tensor = Input((height, width, 1))
    
    #forward path (normal CNN)
    
    c1 = add2Conv2D_blocks(input_tensor, n_filters * 1 , kernel_size = 3, batchnorm=False)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)
    
    c2 = add2Conv2D_blocks(p1, n_filters * 2 , kernel_size = 3, batchnorm=False)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)
    
    c3 = add2Conv2D_blocks(p2, n_filters * 4 , kernel_size = 3, batchnorm=False)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)
    
    c4 = add2Conv2D_blocks(p3, n_filters * 8 , kernel_size = 3, batchnorm=False)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = add2Conv2D_blocks(p4, n_filters * 16 , kernel_size = 3, batchnorm=False)
    
    # reverse path - expansion
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = add2Conv2D_blocks(u6, n_filters * 8, kernel_size = 3, batchnorm = False)
    
    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = add2Conv2D_blocks(u7, n_filters * 4, kernel_size = 3, batchnorm = False)
    
    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = add2Conv2D_blocks(u8, n_filters * 2, kernel_size = 3, batchnorm = False)
    
    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = add2Conv2D_blocks(u9, n_filters * 1, kernel_size = 3, batchnorm = False)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_tensor], outputs=[outputs])
    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])
    
    return model

In [17]:
imgs_train, imgs_mask_train, imgs_test = preprocess_img_data(X_train , y_train , X_test)

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [38]:
ModelCheckpoint?

In [44]:
model = unet_model(image_height, image_width)
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_dice_coef', save_best_only=True, mode='max', verbose=1)
es = keras.callbacks.EarlyStopping(monitor='val_dice_coef',
                              patience=5, mode='max')

print('-'*30)
print('Fitting model...')
print('-'*30)
model.fit(imgs_train, imgs_mask_train, batch_size=32, nb_epoch=100, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint, es])

------------------------------
Fitting model...
------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


Train on 4508 samples, validate on 1127 samples
Epoch 1/100
4508/4508 [==============================] - 40s 9ms/step - loss: -0.0258 - dice_coef: 0.0258 - val_loss: -0.0293 - val_dice_coef: 0.0293
Epoch 2/100
 800/4508 [====>.........................] - ETA: 25s - loss: -0.0292 - dice_coef: 0.0292

KeyboardInterrupt: 

In [19]:
print('-'*30)
print('Loading saved weights...')
print('-'*30)
model.load_weights('weights.h5')

print('-'*30)
print('Predicting masks on test data...')
print('-'*30)
imgs_mask_test = model.predict(imgs_test, verbose=1)
np.save('imgs_mask_test.npy', imgs_mask_test)

------------------------------
Loading saved weights...
------------------------------
------------------------------
Predicting masks on test data...
------------------------------
5508/5508 [==============================] - 17s 3ms/step


In [20]:
print('-' * 30)
print('Saving predicted masks to files...')
print('-' * 30)
pred_dir = 'preds_vinny'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for image, image_id in zip(imgs_mask_test, test_id):
    image = (image[:, :, 0] * 255.).astype(np.uint8)
    imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

------------------------------
Saving predicted masks to files...
------------------------------


/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:140: UserWarning: preds_vinny/3697_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:140: UserWarning: preds_vinny/4402_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:140: UserWarning: preds_vinny/619_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:140: UserWarning: preds_vinny/1741_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:140: UserWarning: preds_vinny/1509_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:140: UserWarning: preds_vinny/2125_pred.png is a low contrast image
  warn('%s is a low contra

In [36]:
def prep(img):
    (image_cols, image_rows) = (580, 420)
    img = img.astype('float32')
    img = (img > 0.5).astype(np.uint8)  # threshold
    img = resize(img, (image_cols, image_rows), preserve_range=True)
    return img


def run_length_enc(label):
    from itertools import chain
    x = label.transpose().flatten()
    y = np.where(x > 0)[0]
    if len(y) < 10:  # consider as empty
        return ''
    z = np.where(np.diff(y) > 1)[0]
    start = np.insert(y[z+1], 0, y[0])
    end = np.append(y[z], y[-1])
    length = end - start
    res = [[s+1, l+1] for s, l in zip(list(start), list(length))]
    res = list(chain.from_iterable(res))
    return ' '.join([str(r) for r in res])


def submission(imgs_id_test):
    imgs_test = np.load('imgs_mask_test.npy')

    argsort = np.argsort(test_id)
    imgs_id_test = np.array(imgs_id_test)[argsort]
    imgs_test = np.array(imgs_test)[argsort]

    total = imgs_test.shape[0]
    ids = []
    rles = []
    for i in range(total):
        img = imgs_test[i]
        img = prep(img)
        rle = run_length_enc(img)

        rles.append(rle)
        ids.append(imgs_id_test[i])

        if i % 100 == 0:
            print('{}/{}'.format(i, total))

    first_row = 'img,pixels'
    file_name = 'submission.csv'

    with open(file_name, 'w+') as f:
        f.write(first_row + '\n')
        for i in range(total):
            s = str(ids[i]) + ',' + rles[i]
            f.write(s + '\n')

In [24]:
imgs_test = np.load('imgs_mask_test.npy')

argsort = np.argsort(test_id)
imgs_id_test = np.array(test_id)[argsort]
imgs_test = np.array(imgs_test)[argsort]

total = imgs_test.shape[0]
ids = []
rles = []
for i in range(total):
    img = imgs_test[i, 0]
    img = prep(img)
    rle = run_length_enc(img)

    rles.append(rle)
    ids.append(imgs_id_test[i])

    break
    if i % 100 == 0:
        print('{}/{}'.format(i, total))

# first_row = 'img,pixels'
# file_name = 'submission.csv'

# with open(file_name, 'w+') as f:
#     f.write(first_row + '\n')
#     for i in range(total):
#         s = str(ids[i]) + ',' + rles[pi]
#         f.write(s + '\n')

In [35]:
imgs_test[0,0].shape

(400, 1)

In [34]:
run_length_enc(prep(imgs_test[0]))

'27481 4 27486 9 28059 21 28639 21 29216 24 29791 29 30368 30 30942 36 31517 41 32096 42 32673 44 33250 47 33827 50 34404 51 34982 53 35559 56 36139 56 36719 56 37297 57 37876 58 38454 58 39034 58 39611 61 40190 59 40768 61 41348 61 41926 62 42506 60 43084 61 43663 59 44241 61 44820 58 45398 58 45977 58 46555 58 47134 54 47713 53 48291 54 48871 54 49451 51 50028 51 50608 48 51187 45 51767 45 52347 40 52927 37 53051 3 53507 35 53627 3 53631 5 54087 32 54206 14 54671 23 54783 21 55253 14 55360 26 55840 6 55938 29 56518 29 57097 35 57674 39 57790 6 58254 39 58367 13 58834 41 58944 22 59411 45 59523 24 59988 50 60101 29 60568 50 60680 32 61148 50 61260 32 61728 50 61840 38 62308 50 62417 45 62888 50 62997 51 63468 48 63576 56 64047 48 64156 65 64627 48 64736 69 65208 45 65316 72 65788 44 65896 73 66370 40 66474 78 66950 39 67054 81 67531 38 67634 81 68111 39 68214 83 68691 39 68793 84 69271 39 69373 81 69851 41 69953 81 70431 41 70530 84 71011 41 71107 87 71591 42 71684 85 72173 40 72215 3

In [37]:
print("Generating submission")
submission(test_id)

Generating submission


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


0/5508
100/5508
200/5508
300/5508
400/5508
500/5508
600/5508
700/5508
800/5508
900/5508
1000/5508
1100/5508
1200/5508
1300/5508
1400/5508
1500/5508
1600/5508
1700/5508
1800/5508
1900/5508
2000/5508
2100/5508
2200/5508
2300/5508
2400/5508
2500/5508
2600/5508
2700/5508
2800/5508
2900/5508
3000/5508
3100/5508
3200/5508
3300/5508
3400/5508
3500/5508
3600/5508
3700/5508
3800/5508
3900/5508
4000/5508
4100/5508
4200/5508
4300/5508
4400/5508
4500/5508
4600/5508
4700/5508
4800/5508
4900/5508
5000/5508
5100/5508
5200/5508
5300/5508
5400/5508
5500/5508
